## Bayesian ANOVA

**Bayesian ANOVA** offers a more flexible and informative alternative to the traditional **frequentist ANOVA**. While classical ANOVA tests whether there are any statistically significant differences between group means, it comes with several limitations:

- It provides only a *p-value*, not the size or uncertainty of effects.
- It assumes normality, homogeneity of variance, and fixed effects.
- It gives no direct probability for hypotheses—just whether the null is rejected or not.

In contrast, **Bayesian ANOVA** uses probability distributions to directly model uncertainty and effect sizes. Instead of asking whether group means are different *in general*, it estimates:

- The **posterior distribution** of each group mean
- The **probability** of differences between groups
- The **credible intervals** that reflect uncertainty in estimates

Bayesian ANOVA can also naturally handle more complex models (e.g., hierarchical structures, unequal variances) and allows for **model comparison using Bayes Factors**, offering a principled way to weigh evidence for competing hypotheses.

In short, **Bayesian ANOVA** goes beyond just testing for significance—it provides a **deeper, probabilistic understanding** of group differences, effect sizes, and model credibility.

In [37]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pymc as pm
import arviz as az
from scipy.stats import gaussian_kde, norm
import ipywidgets as widgets
from IPython.display import display, clear_output
import scipy.stats as st
from scipy.stats import gaussian_kde
from typing import Sequence, Tuple

print("Running with PyMC version:", pm.__version__)

# paths 
# here we have paths to folders where the logging histories of individual training runs are stored.

# for Unet and Swin Transformer models trained on MACS, we have two sets of logs:
unet_macs_loss_dir = r'N:\isipd\projects\p_planetdw\data\methods_test\logs\unet_ae_samples'
swin_macs_loss_dir = r'N:\isipd\projects\p_planetdw\data\methods_test\logs\swin_ae_samples'

# for Unet and Swin Transformer models trained on PS, we have two sets of logs:
unet_ps_loss_dir = r'N:\isipd\projects\p_planetdw\data\methods_test\logs\unet_ps_samples'
swin_ps_loss_dir = r'N:\isipd\projects\p_planetdw\data\methods_test\logs\swin_ps_samples'

# these are the metrics that we want to analyse plus the ones where higher is better.
metrics = ['loss', 'accuracy', 'specificity', 'sensitivity', 'IoU', 'f1_score', 'Hausdorff_distance']
maximize_metrics = {'specificity', 'sensitivity', 'IoU', 'f1_score'}

# and some output directories where we will save the results of our analysis.
unet_macs_output_dir = r'N:\isipd\projects\p_planetdw\data\methods_test\logs\unet_ae_samples'
swin_macs_output_dir = r'N:\isipd\projects\p_planetdw\data\methods_test\logs\swin_ae_samples'

# this function reads the metrics from CSV files in a given directory and returns them as a 3D numpy array, with
# the shape (number of files, number of epochs, number of metrics). It also returns a lookup dictionary for metric names

def read_metrics_as_array(directory, metrics):
    
    """
    Reads CSV files from a directory and extracts specified metrics into a 3D numpy array.
    Args:
        directory (str): Path to the directory containing CSV files.
        metrics (list): List of metric names to extract from the CSV files.
    Returns:
        tuple: A tuple containing:
            - data_array (np.ndarray): A 3D numpy array of shape (num_files, num_epochs, num_metrics).
            - lookup (dict): A dictionary mapping metric names to their indices in the data array.
            - files (list): List of file names processed.
    """

    files = sorted([f for f in os.listdir(directory) if f.endswith('.csv')])
    data_list = []
    
    metric_names = []
    for metric in metrics:
        metric_names.append(metric)
        metric_names.append('val_' + metric)

    for file in files:
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)
        file_data = []

        for name in metric_names:
            if name in df.columns:
                file_data.append(df[name].values)
            else:
                # Fill with NaNs if column is missing
                file_data.append(np.full(len(df), np.nan))
        
        # Transpose so shape is (epochs, metrics)
        file_data = np.stack(file_data, axis=1)  # shape: (epochs, num_metrics)
        data_list.append(file_data)

    # Convert to a 3D array: (files, epochs, metrics)
    data_array = np.stack(data_list, axis=0)

    # Build lookup dict
    lookup = {name: idx for idx, name in enumerate(metric_names)}

    return data_array, lookup, files



# lets import and plot the losses for Unet and Swin Transformer models trained on MACS and PS datasets.

unet_macs, unet_macs_metric_lookup, unet_macs_file_names = read_metrics_as_array(unet_macs_loss_dir, metrics)
swin_macs, swin_macs_metric_lookup, swin_macs_file_names = read_metrics_as_array(swin_macs_loss_dir, metrics)
unet_ps, unet_ps_metric_lookup, unet_ps_file_names = read_metrics_as_array(unet_ps_loss_dir, metrics)
swin_ps, swin_ps_metric_lookup, swin_ps_file_names = read_metrics_as_array(swin_ps_loss_dir, metrics)


# next we will define a function to get the metric values across epochs at the point of the lowest val_loss for each file in the data array.
# we also define a function to help interpret the Bayes factor values.

def get_best_metric(data_array, metric_lookup, metric):
    """
    Get the best metric values across epochs for each file in the data array.
    Args:
        data_array (np.ndarray): 3D array of shape (files, epochs, metrics).
        metric_lookup (dict): Dictionary mapping metric names to their indices.
        metric (str): The metric to evaluate.
        maximize_metrics (set): Set of metrics that should be maximized.
    Returns:
        np.ndarray: Array of best metric values for each file.
    """

    best_values = []


    for i in range(data_array.shape[0]):

        losses = data_array[i, :, metric_lookup['loss']]
        values = data_array[i, :, metric_lookup[metric]]


        best_epoch = np.nanargmin(losses)
        #print(f'best epoch; {best_epoch}')

        best_value = values[best_epoch]
        #print('best value:', best_value)

        best_values.append(best_value)

    return np.array(best_values)



Running with PyMC version: 5.22.0


In [ ]:
def bayes_factor_best(probability_vector_best, best_group_idx, num_groups):

    posterior_probability = probability_vector_best[best_group_idx]
    prior_probability = 1/num_groups # our prior probability assumption is that all groups are equally good

    return(posterior_probability/(1-posterior_probability)) / (prior_probability/(1-prior_probability))

def pairwise_bayes_factor(mu_draws, i, j, tau):

    data = (mu_draws.sel(mu_dim_0=i) - mu_draws.sel(mu_dim_0=j)).values.ravel()
    kde = gaussian_kde(data, bw_method=0.3)
    posterior_density_at_0 = kde.evaluate(0)[0]
    prior_density_at_0 = st.norm.pdf(0, loc=0, scale=np.sqrt(2)*tau)
    return posterior_density_at_0/prior_density_at_0

def model(df, group_col, value_col, diagnostics=True):

    group_code = pd.Categorical(df[group_col]).codes
    unique_groups = pd.Categorical(df[group_col]).categories
    num_groups = len(unique_groups)

    #display(group_code, unique_groups, num_groups)

    with pm.Model() as model:

        # Hyperpriors, same for all groups
        mu_global = pm.Normal('mu_global', 0.5, 0.3) # global mean for all groups
        sigma_global = pm.HalfNormal('sigma_global', 0.2) #global std for all groups

        # group level prior, start of as all the same, informed by global hyperprior
        mu = pm.Normal('mu', mu_global, sigma_global, shape=num_groups) # group mean
        sigma = pm. HalfNormal('sigma', 0.2, shape=num_groups) # group std

        # likelyhoods
        # degrees of freedom
        nu = pm.Exponential('nu', 1/30) + 1

        pm.StudentT('data', mu=mu[group_code], sigma=sigma[group_code], nu=nu, observed=df[value_col].values) # pic mu and sigma for each group

        deltas = {} # store the difference between posterior means here
        for i in range(num_groups):
            for j in range(i+1, num_groups):
                name = f'delta_{i}_{j}'
                deltas[(i, j)] = pm.Deterministic(name, mu[i]-mu[j])

        idata = pm.sample(draws=4000, chains=4, target_accept=0.9, return_inferencedata=True, progressbar=True)

        delta_names = [v.name for v in deltas.values()]

        prior_idata = pm.sample_prior_predictive(var_names=delta_names, return_inferencedata=True, random_seed=99)

        idata.extend(prior_idata) 

        idata = pm.sample_posterior_predictive(trace=idata, var_names=["data"], extend_inferencedata=True,)

    #print diagnostics R‑hat < 1.01 and ESS > 400 are typically “good”
    summary_indv_performance = az.summary(idata, var_names=['mu', 'sigma'])
    #display(summary_indv_performance)

    summary_diff = az.summary(idata, var_names=delta_names)
    #display(summary_diff)

    # draw posteriro probability that each group is best (here lowest/highest mu)
    mu_draws = idata.posterior['mu'] #--> shape of chain x draw x num_groups
    mu_draws_flattened = mu_draws.stack(d=('chain', 'draw'))

    #count which group is best per draw
    if value_col in ['val_loss', 'loss', 'Hausdorff_distance', 'val_Hausdorff_distance']:
        best_count =mu_draws_flattened.argmin(dim='mu_dim_0').values # which group is best per draw, here lowest mu
    
    else:
        best_count= mu_draws_flattened.argmax(dim='mu_dim_0').values # which group is best per draw, here lowest 
    
    prob_best = np.bincount(best_count, minlength=num_groups)/best_count.size
    #display(prob_best)

    #print(f'\nProbability that each group has best {value_col}:')
    for group_idx, probability in enumerate(prob_best):
        #print(f'{unique_groups[group_idx]:20}: {probability:0.3f}')

        BF_best = bayes_factor_best(prob_best, group_idx, num_groups)
        #print(f"BF10 that '{unique_groups[group_idx]}' is the overall best:{BF_best:0.2f}")

    rows = []
    for (i, j), deltavar in deltas.items():
        delta_name = deltavar.name              # e.g. "delta_0_1"
        bf = az.bayes_factor(idata, var_name=delta_name, ref_val=0)["BF10"]

        rows.append({"comparison": f"{unique_groups[i]} vs {unique_groups[j]}","BF10": bf })
    
    bf_table = pd.DataFrame(rows)
    #display(bf_table)


    idx_map = {f"mu[{i}]": unique_groups[i] for i in range(num_groups)}
    perf_table = (
        summary_indv_performance
        .loc[[f"mu[{i}]" for i in range(num_groups)]]     # keep only μ rows
        .rename(index=idx_map)                            # pretty row names
        .reset_index()
        .rename(columns={"index": "group"})
    )

    perf_table["prob_best"] = prob_best
    perf_table["BF10_best"] = [
        bayes_factor_best(prob_best, i, num_groups) for i in range(num_groups)
    ]

    print('\nPerformance of Models according to Posterior\n')
    display(perf_table)

    import re

    def parse_delta(name):
        i, j = map(int, re.findall(r"\d+", name))   # delta_2_3 → (2,3)
        return f"{unique_groups[i]} vs {unique_groups[j]}"

    print('\nDifference in Model Performacne according to Posterior\n')
    diff_table = (
        summary_diff
        .assign(comparison=lambda df_: df_.index.map(parse_delta))
        .reset_index(drop=True)
        .merge(bf_table, on="comparison")           # add the BF10 column
        .loc[:, ["comparison", "mean", "sd", "hdi_3%", "hdi_97%", "BF10"]]
    )

    display(diff_table)
    
    
    az.plot_forest(idata, var_names=["mu"], hdi_prob=0.94, combined=True)

    #az.plot_ppc(idata, num_pp_samples=100)
    #plt.show()

    if diagnostics:
        import seaborn as sns
        sns.set(style="whitegrid")

        # ──────────────────────────
        # Sampler-diagnostics
        # ──────────────────────────
        print("T\nRACE PLOTS → confirm chain mixing and stationarity.")
        az.plot_trace(idata, var_names=["mu", "sigma", "mu_global", "sigma_global", "nu"], legend=True)
        
        plt.show()

        print("\nENERGY PLOT → overlay of energy & histogram; look for overlap (no bimodality).")
        az.plot_energy(idata)
        plt.show()

        print("\nDIVERGENCE PAIRS → red ×’s would mark where NUTS diverged (should be empty).")
        az.plot_pair(idata, var_names=["mu"], kind="kde", divergences=True,
                    coords={"mu_dim_0": np.arange(len(unique_groups))})
        plt.suptitle("Divergences (if any) over μ-space")
        plt.show()

        print("\nPAIR PLOT → examine correlations or multimodality among group means.")
        az.plot_pair(idata, var_names=["mu"], kind="kde", marginals=True)
        plt.suptitle("Joint posterior of group means (μ)"); plt.show()


        # ──────────────────────────
        # Shrinkage check
        # ──────────────────────────
        print("\nSHRINKAGE LINES → how far each empirical mean is pulled toward the hierarchy.")
        raw_means = df.groupby("group")[value_col].mean().rename("empirical_mean")
        posterior_means = perf_table.set_index("group")["mean"].rename("posterior_mean")
        shrink_df = pd.concat([raw_means, posterior_means], axis=1).reset_index()

        plt.figure(figsize=(6, 4))
        for _, row in shrink_df.iterrows():
            plt.plot([0, 1], [row["empirical_mean"], row["posterior_mean"]], marker="o")
        plt.xticks([0, 1], ["empirical", "posterior"])
        plt.title("Shrinkage of group means toward the hierarchy")
        plt.ylabel(value_col)
        for _, row in shrink_df.iterrows():
            plt.text(1.02, row["posterior_mean"], row["group"], va="center")
        plt.tight_layout(); plt.show()


        # ──────────────────────────
        # Probability-of-superiority heat-map
        # ──────────────────────────
        print("\nHEAT-MAP > cell (i,j) = Pr(group i beats group j). 0.5 on diagonal.")
        G = len(unique_groups)
        mu_stacked = (idata.posterior["mu"].stack(sample=("chain", "draw"))
                    .transpose("mu_dim_0", "sample").values)

        if value_col in ['val_loss', 'loss', 'Hausdorff_distance', 'val_Hausdorff_distance']:

            psup = np.array([[0.5 if i==j else (mu_stacked[i] < mu_stacked[j]).mean()
                            for j in range(G)] for i in range(G)])
            
        else:
            psup = np.array([[0.5 if i==j else (mu_stacked[i] > mu_stacked[j]).mean()
                        for j in range(G)] for i in range(G)])

        plt.figure(figsize=(6, 5))
        sns.heatmap(psup, annot=True, fmt=".2f", cmap="Blues",
                    xticklabels=unique_groups, yticklabels=unique_groups)
        plt.title("Pr( μ_row < μ_col )  — smaller val_loss is better"); plt.show()


        # ──────────────────────────
        # Evidence visualisations
        # ──────────────────────────
        print("\nBAR (log BF10) → strength of evidence each group is overall best.")
        plt.figure(figsize=(6, 3))
        sns.barplot(x="group", y="BF10_best", data=perf_table, color="steelblue")
        plt.yscale("log"); plt.ylabel("BF10 (log scale)")
        plt.title("Evidence each group is the overall best")
        plt.xticks(rotation=45, ha="right"); plt.tight_layout(); plt.show()

        print("\nWATERFALL → effect sizes (Δμ) with 95% HDI; crosses zero → weak evidence.")
        sorted_diff = diff_table.sort_values("mean")
        plt.figure(figsize=(6, 4))
        plt.hlines(y=sorted_diff["comparison"], xmin=sorted_diff["hdi_3%"],
                xmax=sorted_diff["hdi_97%"], color="gray")
        plt.scatter(sorted_diff["mean"], sorted_diff["comparison"], color="red")
        plt.axvline(0, color="black", ls="--")
        plt.xlabel("Posterior mean difference Δμ")
        plt.title("Effect sizes and 95% HDI"); plt.tight_layout(); plt.show()

        print("\nLOG-BF HEAT-MAP → positive = row outranks col; magnitude shows evidence.")
        bf_matrix = pd.DataFrame(0.0, index=unique_groups, columns=unique_groups)
        for _, row in diff_table.iterrows():
            g1, g2 = row["comparison"].split(" vs ")
            #bf_matrix.loc[g1, g2] = np.log10(row["BF10"])
            #bf_matrix.loc[g2, g1] = -np.log10(row["BF10"]) #log
            bf_matrix.loc[g1, g2] = row["BF10"] #normal
            bf_matrix.loc[g2, g1] = row["BF10"]  # reciprocal
        plt.figure(figsize=(6, 5))
        sns.heatmap(bf_matrix, annot=True, fmt=".1f", center=0, cmap="coolwarm")
        plt.title("log₁₀ BF10  (positive → row better)"); plt.show()

    return perf_table, diff_table

In [ ]:
tabs = []
tab_titles = []

#metrics = metrics []
#print(metrics)

for metric in metrics: 
    full_metric = 'val_' + metric if not metric.startswith('val_') else metric
    out = widgets.Output()

    with out:
        
        clear_output(wait=True)

        print(f'Processing: {full_metric} at point of best loss\n')

        unet_macs_best = get_best_metric(unet_macs, unet_macs_metric_lookup, full_metric)
        swin_macs_best = get_best_metric(swin_macs, swin_macs_metric_lookup, full_metric)

        unet_ps_best = get_best_metric(unet_ps, unet_ps_metric_lookup, full_metric)
        swin_ps_best = get_best_metric(swin_ps, swin_ps_metric_lookup, full_metric)

        data = pd.concat([
                        pd.DataFrame({full_metric: unet_macs_best, 'group': 'U-Net | Aerial'}),
                        pd.DataFrame({full_metric: swin_macs_best, 'group': 'Swin U-Net | Aerial'}),
                        pd.DataFrame({full_metric: swin_ps_best, 'group': 'Swin U-Net | PS'}),
                        pd.DataFrame({full_metric: unet_ps_best, 'group': 'U-Net | PS'})
                    ]).reset_index(drop=True)

        sns.kdeplot(data, x=full_metric, hue='group')
        plt.show()

        ANOVA(data, 'group', full_metric, diagnostics=True)

    tabs.append(out)
    tab_titles.append(full_metric)

tab_widget = widgets.Tab(children=tabs)

for i, title in enumerate(tab_titles):
    tab_widget.set_title(i, title)

display(tab_widget)

In [33]:
pred = perf_table['mean']
obs = mean_by_group['val_loss_mean']

fit_x = [0,1]
fit_y =[0,1]

display(pred)
display(obs)

plt.scatter(x=obs, y=pred)
plt.plot(fit_x, fit_y)

from sklearn.metrics import mean_squared_error
print('RMSE: ', np.sqrt(mean_squared_error(obs, pred)))


NameError: name 'perf_table' is not defined